In [ ]:
from greenMLAnalysis import greenMLAnalysis
from blueMLAnalysis import blueMLAnalysis
import os
from tensorflow.keras import models, optimizers, layers, metrics

## Making a Neural Network

Use the following code to create a neural network from scratch. I found this architecture to work particularly well for training on green imaging data. Note that you may need to adjust the input shape if the input image changes shape.

In [ ]:
model_path = os.path.join("example_models", "green_model.h5")

In [ ]:
network = models.Sequential()
network.add(layers.Input(shape=(17, 17, 1))) # Usually the input is 17 x 17 px
network.add(layers.RandomFlip())
network.add(layers.RandomBrightness(0.2, (0, 500)))
network.add(layers.RandomContrast(0.2))
network.add(layers.Normalization())
network.add(layers.Conv2D(16, 3, strides=1, activation='relu', padding='same'))
network.add(layers.BatchNormalization())
network.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
network.add(layers.Conv2D(8, 3, strides=1, activation='relu', padding='same'))
network.add(layers.BatchNormalization())
network.add(layers.Flatten())
network.add(layers.Dense(2, activation='sigmoid'))

network.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy',
               metrics=['binary_accuracy'])
network.save(model_path)


# Green Analysis Demo

In [ ]:
data_path = os.path.join("example_data", "green_training")
n_tweezers = 50
n_loops = 20

In [ ]:
analysis = greenMLAnalysis(n_tweezers, n_loops, model_path, make_plots=True, r_atom=2.5)

In [ ]:
analysis.train_model(data_path, model_path=model_path)

In [ ]:
positions, p_db, p_bd, fig = analysis.fidelity_analysis(data_path)

# Blue Analysis Demo

The neural network architecture below seems to work particularly well for blue imaging. 

In [ ]:
model_path = os.path.join("example_models", "blue_model.h5")

In [ ]:
network = models.Sequential()
network.add(layers.Input(shape=(17, 17, 1))) # Usually the input is 17 x 17 px
network.add(layers.RandomFlip())
network.add(layers.Normalization())
network.add(layers.Conv2D(32, 3, strides=1, activation='relu', padding='same'))
network.add(layers.BatchNormalization())
network.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
network.add(layers.Conv2D(16, 3, strides=1, activation='relu', padding='same'))
network.add(layers.BatchNormalization())
network.add(layers.Flatten())
network.add(layers.Dense(2, activation='sigmoid'))

network.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy',
               metrics=['binary_accuracy'])
network.save(model_path)

In [ ]:
data_path = os.path.join("example_data", "blue_training")
green_model_path = os.path.join("example_models", "green_model.h5")
blue_model_path = model_path = os.path.join("example_models", "blue_model.h5")

In [ ]:
analysis = blueMLAnalysis(100, blue_model_path, make_plots=False)

In [ ]:
data_path

In [ ]:
analysis.train_model(data_path, green_model_path)